In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
from skimage.io import imread as imread
from PIL import Image
import imageio


In [ ]:
import tensorflow as tf
from pylab import rcParams

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn import metrics

from sklearn.model_selection import GridSearchCV

In [ ]:
from keras.models import Model, load_model
from keras import optimizers, losses, activations, models

from keras.layers import Input, Dropout, concatenate, GlobalAveragePooling2D
#from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
#from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras import applications
from keras.applications import resnet50
from keras.optimizers import Adam

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!ls -lha ../input/satellite-images-of-hurricane-damage


In [ ]:
!ls -lha ../input/maposm

In [ ]:
input_path = '../input/satellite-images-of-hurricane-damage/'

def print_file_sizes(input_path, subset):
    print('{}:'.format(subset))
    print('')
    path = input_path + subset + '/'
    for f in os.listdir(path):
        if not os.path.isdir(path + f):
            print(f.ljust(30) + str(round(os.path.getsize(path + f) / 1000000, 2)) + 'MB')
        else:
            sizes = [os.path.getsize(path+f+'/'+x)/1000000 for x in os.listdir(path + f)]
            print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + ' ({} files)'.format(len(sizes)))
    print('')
    
print_file_sizes(input_path, 'train_another')
print_file_sizes(input_path, 'validation_another')
print_file_sizes(input_path, 'test_another')
print_file_sizes(input_path, 'test')

In [ ]:
from pathlib import Path
satellite_dir = Path('../input/satellite-images-of-hurricane-damage/')
image_df = pd.DataFrame({'path': list(satellite_dir.glob('**/*.jp*g'))})
image_df.head()

In [ ]:
image_df['damage'] = image_df['path'].map(lambda x: x.parent.stem)
image_df['data_split'] = image_df['path'].map(lambda x: x.parent.parent.stem)
image_df['location'] = image_df['path'].map(lambda x: x.stem)
image_df['lat'] = image_df['location'].map(lambda x: float(x.split('_')[-1]))
image_df['lon'] = image_df['location'].map(lambda x: float(x.split('_')[0]))
image_df.head()

In [ ]:
image_df = image_df.drop(columns=['path', 'location', 'data_split'])

In [ ]:
image_df.head()

In [ ]:
image_df.to_csv('lonlat_houston.csv',index=False)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
for c_group, c_rows in image_df.groupby(['damage']):
    ax1.plot(c_rows['lat'], c_rows['lon'], '.', label=c_group, alpha=0.5)
ax1.legend()
ax1.set_title('Data by Damage Type')
for c_group, c_rows in image_df.groupby(['data_split']):
    ax2.plot(c_rows['lat'], c_rows['lon'], '.', label=c_group, alpha=0.5)
ax2.legend()
ax2.set_title('Data by Group')

In [ ]:
BBox = ((image_df.lon.min(),image_df.lon.max(),image_df.lat.min(), image_df.lat.max()))
BBox

In [ ]:
ruh_m = plt.imread('../input/maposm/map (1).png')


In [ ]:
fig, ax = plt.subplots(figsize = (15,12))
for c_group, c_rows in image_df.groupby(['damage']):
    #ax.plot(c_rows['lon'], c_rows['lat'], '.', label=c_group, alpha=0.5)
    ax.scatter(c_rows['lon'], c_rows['lat'], label=c_group, alpha= 0.5)
ax.legend()
ax.set_title('Data by Damage Type')

ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal')

# Loading Train data

In [ ]:
image_path_damage = input_path + 'train_another/damage/'
image_path_nodamage = input_path + 'train_another/no_damage/'

image_list_damage = os.listdir(image_path_damage)
image_list_damage = [img for img in image_list_damage if not img.startswith('.')] # removing files starting with '.'

image_list_nodamage = os.listdir(image_path_nodamage)
image_list_nodamage = [img for img in image_list_nodamage if not img.startswith('.')] # removing files starting with '.'


len(image_list_damage), len(image_list_nodamage)

In [ ]:
N = 4

img = imageio.imread(image_path_damage + image_list_damage[N])
img2 = imageio.imread(image_path_damage + image_list_damage[N+1])
img3 = imageio.imread(image_path_damage + image_list_damage[N+2])

fig, arr = plt.subplots(1, 3, figsize=(15, 10))
arr[0].imshow(img)
arr[0].set_title('Damage')
arr[1].imshow(img2)
arr[1].set_title('Another Damage')
arr[2].imshow(img3)
arr[2].set_title('Another Damage')

In [ ]:
N = 145
img = imageio.imread(image_path_nodamage + image_list_nodamage[N])
img2 = imageio.imread(image_path_nodamage + image_list_nodamage[N+1])
img3 = imageio.imread(image_path_nodamage + image_list_nodamage[N+2])

fig, arr = plt.subplots(1, 3, figsize=(15, 10))
arr[0].imshow(img)
arr[0].set_title('No Damage')
arr[1].imshow(img2)
arr[1].set_title('Another No Damage')
arr[2].imshow(img3)
arr[2].set_title('Another No Damage')

plt.savefig('/kaggle/working/damage.png')

In [ ]:
img.shape


> ## Plotting RGB bands

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=3, sharex=True, sharey=True, figsize=(8,10))

i = 0

for image in [img, img2, img3]:
    ax[0,i].imshow(image[:,:,0], cmap='Reds')
    ax[0,i].set_title('{} - {}'.format('No Damage'+str(i), 'R'))
    ax[1,i].imshow(image[:,:,1], cmap='Greens')
    ax[1,i].set_title('{} - {}'.format('No Damage'+str(i), 'G'))
    ax[2,i].imshow(image[:,:,2], cmap='Blues')
    ax[2,i].set_title('{} - {}'.format('No Damage'+str(i), 'B'))
    ax[3,i].imshow(image[:,:,2], cmap='Greys_r')
    ax[3,i].set_title('{} - {}'.format('No Damage'+str(i), 'IR'))
    ax[4,i].imshow(image)
    ax[4,i].set_title('{} - {}'.format('No Damage'+str(i), 'RGB'))
    
    i = i+1
    
plt.savefig('/kaggle/working/RGB_noDamage.png', bbox_inches='tight')


In [ ]:
subset = False

In [ ]:
len_images = len(image_list_damage)
IMG_CHANNELS = 3
IMG_HEIGHT, IMG_WIDTH = 128, 128


if subset:

    X_train = np.zeros((4000, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

    y_train = np.zeros((4000, 1), dtype=np.uint8)
    
    for n in range(0, 2000):
    #for n in range(len_images):

        filepath_damage = image_path_damage + image_list_damage[n]

        if filepath_damage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_damage)
            X_train[n] = img
            y_train[n] = 1

        filepath_nodamage = image_path_nodamage + image_list_nodamage[n]

        if filepath_nodamage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_nodamage)
            X_train[2000 + n] = img
            y_train[2000 + n] = 0

else:
    
    X_train = np.zeros((len_images*2, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

    y_train = np.zeros((len_images*2, 1), dtype=np.uint8)

    
    for n in range(len_images):

        filepath_damage = image_path_damage + image_list_damage[n]

        if filepath_damage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_damage)
            X_train[n] = img
            y_train[n] = 1

        filepath_nodamage = image_path_nodamage + image_list_nodamage[n]

        if filepath_nodamage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_nodamage)
            X_train[5000 + n] = img
            y_train[5000 + n] = 0


In [ ]:
X_train.shape, y_train.shape

In [ ]:
np.unique(y_train)

In [ ]:
y_train_list = [i for u in y_train for i in u]
pd.Series(y_train_list).value_counts()

# Validation data

In [ ]:
val_damage = input_path + 'validation_another/damage/'
val_nodamage = input_path + 'validation_another/no_damage/'

val_list_damage = os.listdir(val_damage)
val_list_damage = [img for img in val_list_damage if not img.startswith('.')] # removing files starting with '.'

val_list_nodamage = os.listdir(val_nodamage)
val_list_nodamage = [img for img in val_list_nodamage if not img.startswith('.')] # removing files starting with '.'


len(val_list_damage), len(val_list_nodamage)

In [ ]:
img_val = imageio.imread(val_damage + val_list_damage[0])
img_val.shape

In [ ]:
len_images = len(val_list_damage)
IMG_CHANNELS = 3
IMG_HEIGHT, IMG_WIDTH = 128, 128



if subset:

    X_val = np.zeros((len_images, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    y_val = np.zeros((len_images, 1), dtype=np.uint8)
    
    for n in range(500):

        filepath_damage = val_damage + val_list_damage[n]

        if filepath_damage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_damage)
            X_val[n] = img
            y_val[n] = 1

        filepath_nodamage = val_nodamage + val_list_nodamage[n]

        if filepath_nodamage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_nodamage)
            X_val[500 + n] = img
            y_val[500 + n] = 0
    
else:
    
    X_val = np.zeros((2000, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    y_val = np.zeros((2000, 1), dtype=np.uint8)

    
    for n in range(1000):

        filepath_damage = val_damage + val_list_damage[n]

        if filepath_damage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_damage)
            X_val[n] = img
            y_val[n] = 1

        filepath_nodamage = val_nodamage + val_list_nodamage[n]

        if filepath_nodamage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_nodamage)
            X_val[1000 + n] = img
            y_val[1000 + n] = 0


In [ ]:
X_val.shape, y_val.shape

In [ ]:
np.unique(y_val)

In [ ]:
y_val_list = [i for u in y_val for i in u]
pd.Series(y_val_list).value_counts()

# Test data


> test_another:

> damage                        20.12MB (8000 files)

> no_damage                     3.01MB (1000 files)

In [ ]:
Balanced = True

In [ ]:

if Balanced:
    test_damage = input_path + 'test_another/damage/'
    test_nodamage = input_path + 'test_another/no_damage/'

    test_list_damage = os.listdir(test_damage)
    test_list_damage = [img for img in test_list_damage if not img.startswith('.')] # removing files starting with '.'

    test_list_nodamage = os.listdir(test_nodamage)
    test_list_nodamage = [img for img in test_list_nodamage if not img.startswith('.')] # removing files starting with '.'


else:
    test_damage = input_path + 'test/damage/'
    test_nodamage = input_path + 'test/no_damage/'

    test_list_damage = os.listdir(test_damage)
    test_list_damage = [img for img in test_list_damage if not img.startswith('.')] # removing files starting with '.'

    test_list_nodamage = os.listdir(test_nodamage)
    test_list_nodamage = [img for img in test_list_nodamage if not img.startswith('.')] # removing files starting with '.'


len(test_list_damage), len(test_list_nodamage)

In [ ]:
img_test = imageio.imread(test_damage + test_list_damage[0])
img_test.shape

In [ ]:
len_images = len(test_list_damage)
IMG_CHANNELS = 3
IMG_HEIGHT, IMG_WIDTH = 128, 128



if Balanced:

    X_test = np.zeros((9000, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    y_test = np.zeros((9000, 1), dtype=np.uint8)
    
    for n in range(8000):

        filepath_damage = test_damage + test_list_damage[n]

        if filepath_damage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_damage)
            X_test[n] = img
            y_test[n] = 1
            
    for n in range(1000):
        filepath_nodamage = test_nodamage + test_list_nodamage[n]

        if filepath_nodamage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_nodamage)
            X_test[8000 + n] = img
            y_test[8000 + n] = 0
    
else:
    
    X_test = np.zeros((2000, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    y_test = np.zeros((2000, 1), dtype=np.uint8)

    
    for n in range(1000):

        filepath_damage = test_damage + test_list_damage[n]

        if filepath_damage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_damage)
            X_test[n] = img
            y_test[n] = 1

        filepath_nodamage = test_nodamage + test_list_nodamage[n]

        if filepath_nodamage.split('.')[-1] == 'jpeg':

            #print(filename)
            img = imageio.imread(filepath_nodamage)
            X_test[1000 + n] = img
            y_test[1000 + n] = 0

In [ ]:
X_test.shape, y_test.shape

In [ ]:
np.unique(y_test)

In [ ]:
y_test_list = [i for u in y_test for i in u]
pd.Series(y_test_list).value_counts()

# Data augmentation & Normalization of data with ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=45,
        shear_range=0.2,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)

# Batch processing - Flow from Directory

In [ ]:
num_classes=2
y_train_categorize = to_categorical(y_train, num_classes)
y_test_categorize = to_categorical(y_test, num_classes)
y_val_categorize = to_categorical(y_val, num_classes)

In [ ]:
train_generator = train_datagen.flow(X_train, y_train_categorize, batch_size=32, shuffle=True)

In [ ]:
test_generator = test_datagen.flow(X_test, y_test_categorize, batch_size=1, shuffle=False)

In [ ]:
validation_generator = test_datagen.flow(X_val, y_val_categorize, batch_size=32, shuffle=True)

In [ ]:
y_train.shape, y_train_categorize.shape

In [ ]:
y_test.shape, y_test_categorize.shape

In [ ]:
y_val.shape, y_val_categorize.shape

In [ ]:

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

# PART 1: Custom model

In [ ]:
model_new = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH,3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    # new
    Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH,3)),
    MaxPooling2D(pool_size=(2, 2)),
    # end
    
    Dropout(0.2),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# load weights
#model_new.load_weights("/kaggle/working/Custom_weights.best.hdf5")

# compile
model_new.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_new.summary()


In [ ]:
len(model_new.layers)

> ### Fit model

In [ ]:
# checkpoint
filepath="/kaggle/working/Custom_weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Fit the model
new_history = model_new.fit_generator(
            train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            epochs=75,
            validation_data=validation_generator,
            validation_steps=STEP_SIZE_VALID,
            callbacks=callbacks_list)

#model_new.save('/kaggle/working/hurricane-model-withDropouts-40epochs.h5')



## Reloading best weights

In [ ]:
# load weights
model_new.load_weights("/kaggle/working/Custom_weights.best.hdf5")
# Compile model (required to make predictions)
model_new.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Created model and loaded weights from file")



> ### Learning curves

In [ ]:
plt.figure(1, figsize = (7,9)) 
    
plt.subplot(211)  
plt.plot(new_history.history['accuracy'])  
plt.plot(new_history.history['val_accuracy'])  
plt.title('model accuracy with Custom model')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.ylim(0.3,1)
plt.legend(['train', 'valid']) 
#plt.grid(ls='--', c='C7')
   
plt.subplot(212)  
plt.plot(new_history.history['loss'])  
plt.plot(new_history.history['val_loss'])  
plt.title('model loss with Custom model')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
#plt.grid(ls='--', c='C7')

plt.savefig('/kaggle/working/model_hurricane_Custom_45epochs.png', bbox_inches='tight')

# list all data in some_old_history
#print(history.some_old_history.keys())

In [ ]:
len(test_generator)

### Making predictions

In [ ]:
predictions = model_new.predict_generator(test_generator,steps = STEP_SIZE_TEST)
predictions[:5]

In [ ]:
predicts = np.argmax(predictions, axis=1)
pd.Series(predicts).value_counts()

In [ ]:
y_true = [z for y in y_test for z in y]
pd.Series(y_true).value_counts()

In [ ]:
predicts = pd.Series(predicts)
y_true = pd.Series(y_true)


### Some metrics

In [ ]:
print("Model Custom after 45 epochs \n")
print("Precision", precision_score(y_true, predicts, average=None), '\n')
print("Recall", recall_score(y_true, predicts, average=None), '\n')
print("f1_score", f1_score(y_true, predicts, average=None), '\n')
print("confusion_matrix")
print(confusion_matrix(y_true, predicts))

In [ ]:
matrix = confusion_matrix(y_true, predicts)
df_cm = pd.DataFrame(matrix, 
  index = ['Not damaged', 'Flooded'],
  columns = ['Not damaged', 'Flooded'])

sns.heatmap(df_cm,annot=True,cbar=False, fmt='g', square=True)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix of Custom CNN')

#plt.figure(figsize = (4,7))
#plt.show()
plt.savefig('/kaggle/working/confusion_matrix_Custom.png', bbox_inches='tight')


In [ ]:
val_loss, val_accuracy = model_new.evaluate(validation_generator)
test_loss, test_accuracy = model_new.evaluate(test_generator)

val_accuracy, test_accuracy

# Transfer Learning

Transfer Learning via fine-tuning

* removing the fully-connected layers of an existing network, placing a new set of fully-connected layers on top of the network, and then fine-tuning these weights (and optionally previous layers) to recognize the new object classes
* this technique is usually recommended when the training dataset is large and very similar to the original dataset that the pre-trained model was trained on

## VGG16

In [ ]:

# create the base model from the pre-trained model VGG16
# note that, if using a Kaggle server, internet has to be turned on
weights_used = True


if weights_used:
    pretrained_model = tf.keras.applications.vgg16.VGG16(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                                         include_top=False,
                                                         weights='imagenet')
else:
    pretrained_model = tf.keras.applications.vgg16.VGG16(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                                         include_top=False,
                                                         weights= None)
# freeze the convolutional base
pretrained_model.trainable = False


In [ ]:
#pretrained_model converts each 128x128x3 image to a 4x4x512 block of features. 

pretrained_model.summary()

In [ ]:
len(pretrained_model.layers)

In [ ]:
#Average over the 4x4 spatial locations to convert the block of features into a single 512-element vector per image:

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# set the initializers with a seed for reproducible behaviour
prediction_layer = tf.keras.layers.Dense(2,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=1992),
                                         bias_initializer=tf.keras.initializers.GlorotUniform(seed=1992))
#Now stack all the components using a tf.keras.Sequential model:

vgg_model = tf.keras.Sequential([pretrained_model,
                             global_average_layer,
                             prediction_layer])

base_learning_rate = 0.0001
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

vgg_model.summary()

In [ ]:
len(vgg_model.layers)

In [ ]:

vgg_history = vgg_model.fit_generator(
            train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            epochs=15,
            validation_data=validation_generator,
            validation_steps=STEP_SIZE_VALID)


In [ ]:
plt.figure(1, figsize = (6,10)) 
    
plt.subplot(211)  
plt.plot(vgg_history.history['accuracy'])  
plt.plot(vgg_history.history['val_accuracy'])  
plt.title('model accuracy with vgg16 before fine-tuning')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')
plt.ylim(0.3,1)
plt.legend(['train', 'valid']) 
    
plt.subplot(212)  
plt.plot(vgg_history.history['loss'])  
plt.plot(vgg_history.history['val_loss'])  
plt.title('model loss with vgg16 before fine-tuning')  
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend(['train', 'valid']) 

plt.savefig('/kaggle/working/model_hurricane_vgg16_before_15epochs.png', bbox_inches='tight')

### Some notes from the TensorFlow tutorial on transfer learning:

> Now fine-tune the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic features maps to features associated specifically to our dataset. This should only be attempted after you have trained the top-level classifier with the pre-trained model set to non-trainable. If you add a randomly initialized classifier on top of a pre-trained model and attempt to train all layers jointly, the magnitude of the gradient updates will be too large (due to the random weights from the classifier) and your pre-trained model will forget what it has learned. Only a small number of top layers of the pre-trained model should be fine-tuned. In most convolutional networks, the higher up a layer is, the more specialized it is. The first few layers learn very simple and generic features which generalize to almost all types of images. As you go higher up, the features are increasingly more specific to the dataset on which the model was trained. The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

In [ ]:
# unfreeze the layers
pretrained_model.trainable = True

# let's take a look to see how many layers are in the base model
print("Number of layers in the pre-trained model: ", len(pretrained_model.layers))

In [ ]:
# fine-tune from this layer onwards
fine_tune_at = 15

# freeze all the layers before the `fine_tune_at` layer
for layer in pretrained_model.layers[:fine_tune_at]:
  layer.trainable =  False

#Recompile the model using a lower training rate:

vgg_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/15),
              loss='binary_crossentropy',
              metrics=['accuracy'])

vgg_model.summary()

In [ ]:
len(vgg_model.trainable_variables)


In [ ]:
fine_tune_epochs = 60
initial_epochs = 15
total_epochs =  initial_epochs + fine_tune_epochs


# checkpoint
filepath="/kaggle/working/VGG_weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

vgg_history_final = vgg_model.fit_generator(
            train_generator,
            epochs=total_epochs,
            initial_epoch=vgg_history.epoch[-1]+1,
            validation_data=validation_generator,
            validation_steps=STEP_SIZE_VALID,
            callbacks=callbacks_list)


## Loading best weights

In [ ]:
# load weights
vgg_model.load_weights("/kaggle/working/VGG_weights.best.hdf5")
# Compile model (required to make predictions)
vgg_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Created model and loaded weights from file")



> ### Learning curves

In [ ]:
plt.figure(1, figsize = (7,10)) 
    
plt.subplot(211)  
plt.plot(vgg_history.history['accuracy']+vgg_history_final.history['accuracy']) 
plt.plot(vgg_history.history['val_accuracy']+vgg_history_final.history['val_accuracy'])  
plt.title('model accuracy with vgg16 after fine-tuning')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
plt.ylim(0.3,1)
plt.axvline(initial_epochs, c='C7', ls='--')

plt.subplot(212)  
plt.plot(vgg_history.history['loss']+vgg_history_final.history['loss'])  
plt.plot(vgg_history.history['val_loss']+vgg_history_final.history['val_loss'])  
plt.title('model loss with vgg16 after fine-tuning')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
plt.axvline(initial_epochs, c='C7', ls='--')

plt.savefig('/kaggle/working/model_hurricane_vgg16_after_45epochs.png', bbox_inches='tight')

### Making predictions

In [ ]:
predictions = vgg_model.predict_generator(test_generator,steps = STEP_SIZE_TEST)
predictions[:5]

In [ ]:
predicts = np.argmax(predictions, axis=1)
pd.Series(predicts).value_counts()

In [ ]:
y_true = [z for y in y_test for z in y]
pd.Series(y_true).value_counts()

### Metrics

In [ ]:
predicts = pd.Series(predicts)
y_true = pd.Series(y_true)


In [ ]:
print("Model VGG fine-tuned \n")
print("Precision", precision_score(y_true, predicts, average=None), '\n')
print("Recall", recall_score(y_true, predicts, average=None), '\n')
print("f1_score", f1_score(y_true, predicts, average=None), '\n')
print("confusion_matrix")
print(confusion_matrix(y_true, predicts))

In [ ]:
matrix = confusion_matrix(y_true, predicts)
df_cm = pd.DataFrame(matrix, 
  index = ['Not damaged', 'Flooded'],
  columns = ['Not damaged', 'Flooded'])

sns.heatmap(df_cm,annot=True,cbar=False, fmt='g', square=True)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix of VGG16 CNN')

#plt.figure(figsize = (4,7))
#plt.show()
plt.savefig('/kaggle/working/confusion_matrix_vgg16.png', bbox_inches='tight')


In [ ]:
val_loss, val_accuracy = vgg_model.evaluate(validation_generator)
test_loss, test_accuracy = vgg_model.evaluate(test_generator)

val_accuracy, test_accuracy

# ResNet50

> Tweaking it a bit adding Dropouts 

In [ ]:
resnet_model = resnet50.ResNet50(include_top = False, pooling = 'avg', weights='imagenet')
# freeze the convolutional base
resnet_model.trainable = False

In [ ]:
resnet_model.summary()

In [ ]:
len(resnet_model.layers)

In [ ]:
res_model = Sequential()

res_model.add(resnet_model)
res_model.add(Dense(1024,activation='relu')) #dense layer 2
res_model.add(Dropout(0.2))
res_model.add(Dense(128, activation='relu'))
res_model.add(Dropout(0.5))
res_model.add(Dense(num_classes, activation = 'softmax'))

base_learning_rate = 0.0001
res_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

res_model.summary()


In [ ]:
len(res_model.layers)

In [ ]:
res_history = res_model.fit_generator(
            train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            epochs=10,
            validation_data=validation_generator,
            validation_steps=STEP_SIZE_VALID)


In [ ]:
plt.figure(1, figsize = (6,10)) 
    
plt.subplot(211)  
plt.plot(res_history.history['accuracy'])  
plt.plot(res_history.history['val_accuracy'])  
plt.title('model accuracy with ResNet50 before fine-tuning')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')
plt.ylim(0.3,1)
plt.legend(['train', 'valid']) 
    
plt.subplot(212)  
plt.plot(res_history.history['loss'])  
plt.plot(res_history.history['val_loss'])  
plt.title('model loss with ResNet50 before fine-tuning')  
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend(['train', 'valid']) 

plt.savefig('/kaggle/working/model_hurricane_resnet50_before_15epochs.png', bbox_inches='tight')

In [ ]:
# unfreeze the layers
resnet_model.trainable = True

# let's take a look to see how many layers are in the base model
print("Number of layers in the pre-trained model: ", len(resnet_model.layers))

# fine-tune from this layer onwards
fine_tune_at = 10

# freeze all the layers before the `fine_tune_at` layer
for layer in resnet_model.layers[:fine_tune_at]:
  layer.trainable =  False

#Recompile the model using a lower training rate:

res_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/10),
              loss='binary_crossentropy',
              metrics=['accuracy'])

res_model.summary()

In [ ]:
#len(res_model.trainable_variables)


In [ ]:
initial_epochs = 10
fine_tune_epochs = 65
total_epochs =  initial_epochs + fine_tune_epochs

# checkpoint
filepath="/kaggle/working/ResNet50_weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


res_history_final = res_model.fit_generator(
            train_generator,
            epochs=total_epochs,
            initial_epoch=res_history.epoch[-1]+1,
            validation_data=validation_generator,
            validation_steps=STEP_SIZE_VALID,
            callbacks=callbacks_list)


### Loading weights

In [ ]:
input_path = '../input/resnet50-weights/'

for f in os.listdir(input_path):
    print(f)


In [ ]:
# load weights
res_model.load_weights("../input/resnet50-weights/ResNet50_weights.best.hdf5")

# Compile model (required to make predictions)
res_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/10), metrics=['accuracy'])
print("Created model and loaded weights from file")



In [ ]:
plt.figure(1, figsize = (7,10)) 
    
plt.subplot(211)  
plt.plot(res_history.history['accuracy']+res_history_final.history['accuracy'])  
plt.plot(res_history.history['val_accuracy']+res_history_final.history['val_accuracy'])  
plt.title('model accuracy with ResNet50 after fine-tuning')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')
plt.ylim(0.3,1.03)
plt.legend(['train', 'valid']) 
plt.axvline(initial_epochs, c='C7', ls='--')
    
plt.subplot(212)  
plt.plot(res_history.history['loss']+res_history_final.history['loss'])  
plt.plot(res_history.history['val_loss']+res_history_final.history['val_loss'])  
plt.title('model loss with ResNet50 after fine-tuning')  
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend(['train', 'valid']) 
plt.axvline(initial_epochs, c='C7', ls='--')

plt.savefig('/kaggle/working/model_hurricane_resnet50_after_45epochs.png', bbox_inches='tight')


### Making predictions

In [ ]:
predictions = res_model.predict_generator(test_generator,steps = STEP_SIZE_TEST)

predicts = np.argmax(predictions, axis=1)
pd.Series(predicts).value_counts()

In [ ]:
y_true = [z for y in y_test for z in y]
pd.Series(y_true).value_counts()

In [ ]:
predicts = pd.Series(predicts)
y_true = pd.Series(y_true)

### Metrics

In [ ]:
print("Model ResNet50 fine-tuned \n")
print("Precision", precision_score(y_true, predicts, average=None), '\n')
print("Recall", recall_score(y_true, predicts, average=None), '\n')
print("f1_score", f1_score(y_true, predicts, average=None), '\n')
print("confusion_matrix")
print(confusion_matrix(y_true, predicts))

In [ ]:
matrix = confusion_matrix(y_true, predicts)
df_cm = pd.DataFrame(matrix, 
  index = ['Not damaged', 'Flooded'],
  columns = ['Not damaged', 'Flooded'])

sns.heatmap(df_cm,annot=True,cbar=False, fmt='g', square=True)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix of ResNet50 CNN')

#plt.figure(figsize = (4,7))
#plt.show()
plt.savefig('/kaggle/working/confusion_matrix_resnet50.png', bbox_inches='tight')


In [ ]:
val_loss, val_accuracy = res_model.evaluate(validation_generator)
test_loss, test_accuracy = res_model.evaluate(test_generator)

val_accuracy, test_accuracy

In [ ]:
plt.figure(figsize=(5,2))
plt.title('Label distribution in unbalanced test data')
pd.DataFrame(y_test_plot)['label'].value_counts().plot(kind='barh')
plt.savefig('/kaggle/working/distribution.png', bbox_inches='tight')


In [ ]:
y_test_plot = pd.DataFrame(y_test)
y_test_plot.columns = ['label']
y_test_plot.head()

# ResNet152

In [ ]:

resnet152_model = ResNet152(include_top = False, pooling = 'avg', weights='imagenet')

# freeze the convolutional base
resnet152_model.trainable = False

In [ ]:
len(resnet152_model.layers)

In [ ]:
res152_model = Sequential()

res152_model.add(resnet152_model)
res152_model.add(Dense(1024,activation='relu')) #dense layer 2
res152_model.add(Dropout(0.2))
res152_model.add(Dense(128, activation='relu'))
res152_model.add(Dropout(0.5))
res152_model.add(Dense(num_classes, activation = 'softmax'))

base_learning_rate = 0.0001
res152_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

res152_model.summary()


In [ ]:
res152_history = res152_model.fit_generator(
            train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            epochs=10,
            validation_data=validation_generator,
            validation_steps=STEP_SIZE_VALID)


In [ ]:
# unfreeze the layers
resnet152_model.trainable = True

# let's take a look to see how many layers are in the base model
print("Number of layers in the pre-trained model: ", len(resnet152_model.layers))

# fine-tune from this layer onwards
fine_tune_at = 10

# freeze all the layers before the `fine_tune_at` layer
for layer in resnet152_model.layers[:fine_tune_at]:
  layer.trainable =  False

#Recompile the model using a lower training rate:

res152_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/10),
              loss='binary_crossentropy',
              metrics=['accuracy'])

res152_model.summary()

In [ ]:
initial_epochs = 10
fine_tune_epochs = 65
total_epochs =  initial_epochs + fine_tune_epochs

# checkpoint
filepath="/kaggle/working/ResNet152_weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


res152_history_final = res152_model.fit_generator(
            train_generator,
            epochs=total_epochs,
            initial_epoch=res152_history.epoch[-1]+1,
            validation_data=validation_generator,
            validation_steps=STEP_SIZE_VALID,
            callbacks=callbacks_list)


### Loading weights

In [ ]:
# load weights
res152_model.load_weights("/kaggle/working/ResNet101_weights.best.hdf5")
# Compile model (required to make predictions)
res152_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/10), metrics=['accuracy'])
print("Created model and loaded weights from file")



In [ ]:
plt.figure(1, figsize = (7,10)) 
    
plt.subplot(211)  
plt.plot(res152_history.history['accuracy']+res152_history_final.history['accuracy'])  
plt.plot(res152_history.history['val_accuracy']+res152_history_final.history['val_accuracy'])  
plt.title('model accuracy with ResNet152 after fine-tuning')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')
plt.ylim(0.3,1.03)
plt.legend(['train', 'valid']) 
plt.axvline(initial_epochs, c='C7', ls='--')
    
plt.subplot(212)  
plt.plot(res152_history.history['loss']+res152_history_final.history['loss'])  
plt.plot(res152_history.history['val_loss']+res152_history_final.history['val_loss'])  
plt.title('model loss with ResNet152 after fine-tuning')  
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend(['train', 'valid']) 
plt.axvline(initial_epochs, c='C7', ls='--')

plt.savefig('/kaggle/working/model_hurricane_resnet152_after_45epochs.png', bbox_inches='tight')


In [ ]:
predictions = res152_model.predict_generator(test_generator,steps = STEP_SIZE_TEST)

predicts = np.argmax(predictions, axis=1)
pd.Series(predicts).value_counts()

In [ ]:
y_true = [z for y in y_test for z in y]
pd.Series(y_true).value_counts()

In [ ]:
predicts = pd.Series(predicts)
y_true = pd.Series(y_true)

In [ ]:
print("Model ResNet101 fine-tuned on unbalanced Test \n")
print("Precision", precision_score(y_true, predicts, average=None), '\n')
print("Recall", recall_score(y_true, predicts, average=None), '\n')
print("f1_score", f1_score(y_true, predicts, average=None), '\n')
print("confusion_matrix")
print(confusion_matrix(y_true, predicts))

In [ ]:
matrix = confusion_matrix(y_true, predicts)
df_cm = pd.DataFrame(matrix, 
  index = ['Not damaged', 'Flooded'],
  columns = ['Not damaged', 'Flooded'])

sns.heatmap(df_cm,annot=True,cbar=False, fmt='g', square=True)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix of ResNet152 model')

#plt.figure(figsize = (4,7))
#plt.show()
plt.savefig('/kaggle/working/confusion_matrix_resnet101.png', bbox_inches='tight')


In [ ]:
val_loss, val_accuracy = res152_model.evaluate(validation_generator)
test_loss, test_accuracy = res152_model.evaluate(test_generator)

val_accuracy, test_accuracy